## 离散化和面元划分
`cut`  
将连续数据（样本）划分为“面元”（bin）  
参数  
`right`：面元区间是否包括右侧数值，默认`True`包括右侧数值但不包括左侧数值  
`labels`：设置面元区间名称，接收包含区间名称的列表或数组  
`precision`：面元区间边界数值的小数位数
- 样本应该为一个序列如array或list
- 可以传入一个面元，面元是一个序列如list或array，其每个元素之间就是一个区间，样本根据每个区间进行分类
- 也可以传入一个代表面元数量的整数，会根据面元数量及样本的最大最小值计算出等长面元，然后根据这个面元进划分，`precision`参数可以设置面元区间的数值小数位数
- 划分后返回一个Categorical对象
- 面元的每个区间默认包括最右侧数值而不包括左侧数值，因此默认情况下**面元序列最左侧数值相同的数值是不参与划分的**；可使用`right=False`修改为左侧闭端
  
`Categorical`对象：  
属性  
`codes`：array类型，样本每个元素所在区间的序号  
`categories`：分类区间信息，包括区间列表，闭端方向，数据类型等

In [1]:
import numpy as np
import pandas as pd

In [2]:
score = [6, 1, 7, 8, 7, 8, 4, 5, 9]
score

[6, 1, 7, 8, 7, 8, 4, 5, 9]

In [3]:
bins = [1, 6, 8, 10]  # 定义面元序列
cats = pd.cut(score, bins)  # 开始划分

In [4]:
# 返回的是Categorical对象，可以看到每个score元素所属的区间，以及每个区间的范围
# 注意划分点的数值6和8分别属于区间0和区间1
# 1在面元序列内，但是最左侧数据，默认是不包括最左侧数据的，因此1不在面元范围内，表示为NaN
cats

[(1, 6], NaN, (6, 8], (6, 8], (6, 8], (6, 8], (1, 6], (1, 6], (8, 10]]
Categories (3, interval[int64]): [(1, 6] < (6, 8] < (8, 10]]

In [5]:
# codes属性表示score的每个元素所在的区间序号，-1代表数值不在面元范围内
cats.codes

array([ 0, -1,  1,  1,  1,  1,  0,  0,  2], dtype=int8)

In [6]:
cats.categories  # categories属性，分类区间信息

IntervalIndex([(1, 6], (6, 8], (8, 10]]
              closed='right',
              dtype='interval[int64]')

In [7]:
# value_counts()对每个区间的元素计数，和后面的qcut()结果比较
pd.value_counts(cats)

(6, 8]     4
(1, 6]     3
(8, 10]    1
dtype: int64

In [8]:
# 设置为不包含右侧（包含左侧），并为每个区间设置名称
# 注意6和8分别属于区间1和区间2，而1也在面元范围内属于区间0
pd.cut(score, bins, right=False, labels=['bad', 'ok', 'good'])

[ok, bad, ok, good, ok, good, bad, bad, good]
Categories (3, object): [bad < ok < good]

In [9]:
# 传入面元数量3，根据score最大最小值，计算出等长的3个区间，并设置小数位数为2
pd.cut(score, 3, precision=2)

[(3.67, 6.33], (0.99, 3.67], (6.33, 9.0], (6.33, 9.0], (6.33, 9.0], (6.33, 9.0], (3.67, 6.33], (3.67, 6.33], (6.33, 9.0]]
Categories (3, interval[float64]): [(0.99, 3.67] < (3.67, 6.33] < (6.33, 9.0]]

In [10]:
# qcut()根据样本分布情况划分
cats_q = pd.qcut(score, 3, precision=0)
cats_q

[(6.0, 7.0], (0.0, 6.0], (6.0, 7.0], (7.0, 9.0], (6.0, 7.0], (7.0, 9.0], (0.0, 6.0], (0.0, 6.0], (7.0, 9.0]]
Categories (3, interval[float64]): [(0.0, 6.0] < (6.0, 7.0] < (7.0, 9.0]]

`qcut`  
类似于`cut`，根据样本的分布情况进行划分
- 传入区间数量时，划分出的每个区间的样本数量相同  
- 传入分位数（列表或数组，元素为0到1之间的数值，相当于百分位，包含端点）时，每个区间的样本数量按分位数的比例进行划分

In [11]:
# qcut()划分出来的区间每个元素数量相同
pd.value_counts(cats_q)

(7.0, 9.0]    3
(6.0, 7.0]    3
(0.0, 6.0]    3
dtype: int64

In [12]:
# 传入分位数来划分，比如9个样本，2/3为一个区间，另外1/3为一个区间
pd.value_counts(pd.qcut(score, [0, 2/3, 1]))

(0.999, 7.333]    6
(7.333, 9.0]      3
dtype: int64